#Dewrapping lines

In [0]:
#get authorization from google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls "/content/drive/My Drive/cd_processing/Data/hnyc/cleaning/1880/nypl_1880_81_starred.txt"

'/content/drive/My Drive/cd_processing/Data/hnyc/cleaning/1880/nypl_1880_81_starred.txt'


In [0]:
#load data
import glob
file_path = "/content/drive/My Drive/cd_processing/Data/hnyc/cleaning/1880/nypl_1880_81_starred.txt"
with open(file_path) as f:
  data = f.readlines()  

##Divide records into different sections based on first letter

In [0]:
d = {}
value = []
i = 0
for line in data:
  line = line.strip()
  if line:
    if len(line) < 6:
      if line[-3:] == "***":
        # save previous value
        if value:
          if value[0] == "'":
            value = value[1:]
          d[key] = value
        
        key = line
        value = []
    else:
      value.append(line)

##Combline lines that belong to the same record

Criteria:

1) First letter of next line not equal to the initial letter in this section

2) Special cases in the start of next line: B’klyn, B'way, E., W.

In [0]:
for key,value in d.items():
  initial = key[0]
  new_value = []
  n = len(value)
  i = 0
  
  while i < n:
    if i < n-1:
      if (value[i+1][0] != initial and value[i+1][0] != "*") or (value[i+1][0:2] in ["B’", "E.", "W."]) or (value[i+1][0:2] == ". "):
        if value[i][-1] == "-":
          s = value[i][:-1] + value[i+1]
        else:
          s = value[i] + ' ' + value[i+1]
        new_value.append(s)
        i += 2
      else:
        new_value.append(value[i])
        i += 1 
    else:
      if value[i][0] == initial:
        new_value.append(value[i])
      i += 1
  
  d[key] = new_value      

In [0]:
#remove "(see adv...."
for key,value in d.items():
  n = len(value)
  for i in range(n):
    ls = value[i].split()
    n2 = len(ls)
    for j in range(n2):
      if "see" in ls[j]:
        value[i] = ' '.join(ls[:j])
        break

##Identify business vs person records

In [0]:
# add an index to each record
# in another dictionary, for each index specify whether it's business or person record
index = 1
d_bp = []
for key,value in d.items():
  n = len(value)
  for i in range(n):
    ls = value[i].split()
    if ls:
      if ls[0][0] == "*" or (len(ls[0]) > 1 and ls[0].isupper()):
        value[i] = value[i][1:]
        d_bp.append([str(index), "b"])
      else:
        d_bp.append([str(index), "p"])
    
      value[i] = str(index) + "\t" + value[i]
      index += 1

##Save files

In [0]:
file = open("/content/drive/My Drive/cd_processing/Data/hnyc/cleaning/1880/nypl_1880_81_starred_clean.txt", "w")
for key,value in d.items():
  file.write(key + '\n')
  for record in value:
    file.write(record + '\n')
  file.write('\n')

file.close()

file2 = open("/content/drive/My Drive/cd_processing/Data/hnyc/cleaning/1880/index_business_person.txt", "w")
for item in d_bp:
  file2.write(item[0] + ' ' + item[1] + '\n')
  
file2.close()